In [1]:
import pandas as pd

In [201]:
df_full = pd.read_excel('InternationalDeclarations.xlsx')
df_full = df_full[['id-trace','concept:name','time:timestamp']]
df_full.columns = ['case:concept:name', 'concept:name', 'time:timestamp']
df_full['time:timestamp'] = pd.to_datetime(df_full['time:timestamp'])
df_full = df_full.sort_values(['case:concept:name','time:timestamp'], ascending=[True,True])
df_full = df_full.reset_index(drop=True)

In [202]:
df_full.head()

,case:concept:name,concept:name,time:timestamp
0,declaration 1002,Permit SUBMITTED by EMPLOYEE,2018-03-01 09:55:17
1,declaration 1002,Permit APPROVED by ADMINISTRATION,2018-03-01 09:55:21
2,declaration 1002,Permit FINAL_APPROVED by SUPERVISOR,2018-03-01 14:01:48
3,declaration 1002,Start trip,2018-03-18 23:00:00
4,declaration 1002,End trip,2018-03-22 23:00:00


In [ ]:
# Создадим словарь объединения событий и примапим его

In [110]:
_dict = {'Permit SUBMITTED by EMPLOYEE': 'Permit SUBMITTED',
 'Permit APPROVED by ADMINISTRATION': 'Permit APPROVED',
 'Permit APPROVED by BUDGET OWNER': 'Permit APPROVED',
 'Permit APPROVED by PRE_APPROVER': 'Permit APPROVED',
 'Permit APPROVED by SUPERVISOR': 'Permit APPROVED',
 'Permit FINAL_APPROVED by DIRECTOR': 'Permit FINAL_APPROVED',
 'Permit FINAL_APPROVED by SUPERVISOR': 'Permit FINAL_APPROVED',
 'Start trip': 'Start trip',
 'End trip': 'End trip',
 'Permit REJECTED by ADMINISTRATION': 'Permit REJECTED',
 'Permit REJECTED by BUDGET OWNER': 'Permit REJECTED',
 'Permit REJECTED by DIRECTOR': 'Permit REJECTED',
 'Permit REJECTED by EMPLOYEE': 'Permit REJECTED',
 'Permit REJECTED by MISSING': 'Permit REJECTED',
 'Permit REJECTED by PRE_APPROVER': 'Permit REJECTED',
 'Permit REJECTED by SUPERVISOR': 'Permit REJECTED',
 'Declaration SUBMITTED by EMPLOYEE': 'Declaration SUBMITTED',
 'Declaration SAVED by EMPLOYEE': 'Declaration SAVED',
 'Declaration APPROVED by ADMINISTRATION': 'Declaration APPROVED',
 'Declaration APPROVED by BUDGET OWNER': 'Declaration APPROVED',
 'Declaration APPROVED by PRE_APPROVER': 'Declaration APPROVED',
 'Declaration APPROVED by SUPERVISOR': 'Declaration APPROVED',
 'Declaration FINAL_APPROVED by DIRECTOR': 'Declaration FINAL_APPROVED',
 'Declaration FINAL_APPROVED by SUPERVISOR': 'Declaration FINAL_APPROVED',
 'Declaration REJECTED by ADMINISTRATION': 'Declaration REJECTED',
 'Declaration REJECTED by BUDGET OWNER': 'Declaration REJECTED',
 'Declaration REJECTED by DIRECTOR': 'Declaration REJECTED',
 'Declaration REJECTED by EMPLOYEE': 'Declaration REJECTED',
 'Declaration REJECTED by MISSING': 'Declaration REJECTED',
 'Declaration REJECTED by PRE_APPROVER': 'Declaration REJECTED',
 'Declaration REJECTED by SUPERVISOR': 'Declaration REJECTED',
 'Request Payment': 'Request Payment',
 'Payment Handled': 'Payment Handled',
 'Send Reminder': 'Send Reminder'}

df_full_gr = df_full.copy()
df_full_gr['concept:name'] = df_full_gr['concept:name'].map(_dict)

In [ ]:
# Получим варианты

In [111]:
variant = df_full_gr.groupby('case:concept:name')['concept:name'].agg(lambda col: ' -> '.join(col))
df_variant = pd.DataFrame(variant)
df_variant.columns = ['variant']

In [ ]:
# Выведем на экран несколько цепочек

In [226]:
for row in df_variant['variant'][6:10]:
    print(row)
    print('')

Permit SUBMITTED -> Permit APPROVED -> Permit FINAL_APPROVED -> Start trip -> Declaration SUBMITTED -> End trip -> Declaration REJECTED -> Declaration REJECTED -> Declaration SUBMITTED -> Declaration APPROVED -> Declaration FINAL_APPROVED -> Request Payment -> Payment Handled

Permit SUBMITTED -> Permit APPROVED -> Permit APPROVED -> Permit FINAL_APPROVED -> Start trip -> End trip -> Send Reminder -> Declaration SUBMITTED -> Declaration APPROVED -> Declaration APPROVED -> Declaration FINAL_APPROVED -> Request Payment -> Payment Handled

Permit SUBMITTED -> Permit APPROVED -> Permit FINAL_APPROVED -> Start trip -> End trip -> Declaration SUBMITTED -> Declaration APPROVED -> Declaration FINAL_APPROVED -> Request Payment -> Payment Handled

Permit SUBMITTED -> Permit APPROVED -> Permit APPROVED -> Permit FINAL_APPROVED -> Start trip -> End trip -> Declaration SUBMITTED -> Declaration APPROVED -> Declaration APPROVED -> Declaration FINAL_APPROVED -> Request Payment -> Payment Handled



In [227]:
#Очевидно что с такими вариантами работать неудобно, поэтому прибегнем к использованию "слов процесса"
import string
graph_word_dict = {}
for index,x in enumerate(df_full_gr['concept:name'].value_counts().keys()):
    graph_word_dict[x] = string.ascii_uppercase[index]
    print(string.ascii_uppercase[index]+' '+x)
df_full_gr['short:concept:name'] = df_full_gr['concept:name'].map(graph_word_dict)

A Declaration SUBMITTED
B Permit APPROVED
C Declaration APPROVED
D End trip
E Start trip
F Declaration FINAL_APPROVED
G Permit SUBMITTED
H Payment Handled
I Request Payment
J Permit FINAL_APPROVED
K Declaration REJECTED
L Permit REJECTED
M Send Reminder
N Declaration SAVED


In [ ]:
# Получим варианты

In [228]:
variant_short = df_full_gr.groupby('case:concept:name')['short:concept:name'].agg(lambda col: ''.join(col))
df_variant_short = pd.DataFrame(variant_short)
df_variant_short.columns = ['variant']

In [231]:
df_variant_short['variant'].value_counts()

GBJEDACFIH                     1689
GBBJEDACCFIH                    652
GJEDAFIH                        361
GBJEDAKKACFIH                   350
GBBJEDAKKACCFIH                 208
EDGBJACFIH                      178
GBBJEDACFIH                     127
ACFIHED                         126
AFIHED                          111
GBJEDMACFIH                      94
ACCFIHED                         87
GBEJDACFIH                       86
GBBBJEDACCFIH                    64
GJEDACFIH                        64
GBJEDACCFIH                      63
EDGBBJACCFIH                     63
GBJEDAKKAKKACFIH                 52
GBJEDAFIH                        51
GBBJEDMACCFIH                    49
GBJEDAKK                         48
GBEDJACFIH                       47
GBBJEDAKKAKKACCFIH               42
EGBJACFIHD                       37
EDGJAFIH                         36
GBBJEDAKKACFIH                   36
GBBEJDACCFIH                     34
EDGBJAKKACFIH                    31
GBJEDACKKACFIH              

In [235]:
#Если заранее составить перечень корректных вариантов развития процесса, без отклонений от ВНД, то возможно выполнение мониторинга вариантов процесса с отклонениями

In [242]:
no_dev = ['GBJEDACFIH','GBBJEDACCFIH','GJEDAFIH','GBJEDAKKACFIH','GBBJEDACFIH',
        'EDGBJACFIH','EDGBBJACCFIH','ACFIHED','GBJEDMACFIH','GJEDACFIH']

In [243]:
df_variant_short[~df_variant_short['variant'].isin(no_dev)]['variant'].value_counts()

GBBJEDAKKACCFIH         208
AFIHED                  111
ACCFIHED                 87
GBEJDACFIH               86
GBBBJEDACCFIH            64
GBJEDACCFIH              63
GBJEDAKKAKKACFIH         52
GBJEDAFIH                51
GBBJEDMACCFIH            49
GBJEDAKK                 48
GBEDJACFIH               47
GBBJEDAKKAKKACCFIH       42
EGBJACFIHD               37
EDGJAFIH                 36
GBBJEDAKKACFIH           36
GBBEJDACCFIH             34
EDGBJAKKACFIH            31
GBJEDACKKACFIH           31
EGBJDACFIH               28
GBBEDJACCFIH             27
AKKACFIHED               27
GBJEDACFKACFIH           27
EDGBJMACFIH              25
GBJEDMAKKACFIH           23
GJEDAFKAFIH              22
GLLGBJEDACFIH            22
GBJAEDCFIH               21
GBEDBJACCFIH             21
GBJEADCFIH               20
GBJEACDFIH               19
                       ... 
GBLLGBBJEDACFIH           1
AKKACFIEHD                1
GBLLGBBBJEDAKKACCFIH      1
GEBBJDAKKACCFIH           1
EDGBJAKKAKKACFIH    

In [248]:
list = df_variant_short[~df_variant_short['variant'].isin(no_dev)].index

In [249]:
list

Index(['declaration 10069', 'declaration 10089', 'declaration 10106',
       'declaration 10168', 'declaration 10279', 'declaration 1041',
       'declaration 10455', 'declaration 10472', 'declaration 10495',
       'declaration 10522',
       ...
       'declaration 9732', 'declaration 9750', 'declaration 9762',
       'declaration 9780', 'declaration 9801', 'declaration 9838',
       'declaration 9856', 'declaration 9918', 'declaration 9968',
       'declaration 9995'],
      dtype='object', name='case:concept:name', length=2745)

In [250]:
df_full[df_full['case:concept:name'].isin(list)].head()

,case:concept:name,concept:name,time:timestamp
44,declaration 10069,Permit SUBMITTED by EMPLOYEE,2018-02-21 15:28:17
45,declaration 10069,Permit APPROVED by ADMINISTRATION,2018-02-21 15:28:19
46,declaration 10069,Start trip,2018-02-21 23:00:00
47,declaration 10069,End trip,2018-02-21 23:00:00
48,declaration 10069,Permit FINAL_APPROVED by SUPERVISOR,2018-02-22 07:27:05
